In [16]:
import datetime
import pandas as pd
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG
import talib
import yfinance as yf

In [17]:
aapl = yf.Ticker("aapl")
aapl_historical = aapl.history(start="2020-06-02", end="2023-06-07")
aapl_historical

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-06-02 00:00:00-04:00,78.312329,78.969104,77.867968,78.944687,87642800,0.0,0.0
2020-06-03 00:00:00-04:00,79.266956,79.642955,78.690749,79.379265,104491200,0.0,0.0
2020-06-04 00:00:00-04:00,79.201056,79.501360,78.319657,78.695656,87560400,0.0,0.0
2020-06-05 00:00:00-04:00,78.947137,80.998027,78.917839,80.936989,137250400,0.0,0.0
2020-06-08 00:00:00-04:00,80.631788,81.449705,79.916419,81.415520,95654400,0.0,0.0
...,...,...,...,...,...,...,...
2023-05-31 00:00:00-04:00,176.393224,178.402557,175.826227,176.313644,99625300,0.0,0.0
2023-06-01 00:00:00-04:00,176.761268,179.168482,175.995331,179.138641,68901800,0.0,0.0
2023-06-02 00:00:00-04:00,180.073674,180.819712,178.313020,179.994095,61945900,0.0,0.0


In [18]:
def MeanReversionStrat(data, range):
    class MeanReversion(Strategy):

        # Do as much initial computation as possible
        def init(self):
            self.sma = self.I(talib.SMA, self.data.Close, range)


        # Step through bars one by one
        # Note that multiple buys are a thing here

        def next(self):
            if crossover(self.data.Close, self.sma):
                self.position.close()
            elif crossover(self.sma, self.data.Close):
                self.buy()


    bt = Backtest(data, MeanReversion, cash=10_000)
    stats =  bt.run()
    print(stats)
    

In [19]:
def BollingerStart(data, range, upperbound, lowerbound):
    class BollingerReversion(Strategy):
        # Do as much initial computation as possible
        def init(self):
            self.bands = self.I(talib.BBANDS, self.data.Close, range, upperbound, lowerbound)

        # Step through bars one by one
        # Note that multiple buys are a thing here

        def next(self):
            if crossover(self.data.Close, self.bands[2][-1]):
                self.position.close()
            elif crossover(self.bands[0][-1], self.data.Close):
                self.buy()


    bt = Backtest(data, BollingerReversion, cash=10_000)
    stats =  bt.run()
    print(stats)


In [20]:
MeanReversionStrat(aapl_historical,20)

Start                     2020-06-02 00:00...
End                       2023-06-06 00:00...
Duration                   1099 days 00:00:00
Exposure Time [%]                   43.346509
Equity Final [$]                 10798.199407
Equity Peak [$]                  12374.567146
Return [%]                           7.981994
Buy & Hold Return [%]              125.807838
Return (Ann.) [%]                    2.582475
Volatility (Ann.) [%]               22.267664
Sharpe Ratio                         0.115974
Sortino Ratio                        0.173934
Calmar Ratio                         0.096829
Max. Drawdown [%]                  -26.670583
Avg. Drawdown [%]                   -7.614612
Max. Drawdown Duration      844 days 00:00:00
Avg. Drawdown Duration      162 days 00:00:00
# Trades                                   36
Win Rate [%]                        72.222222
Best Trade [%]                       3.898148
Worst Trade [%]                     -8.662729
Avg. Trade [%]                    

In [21]:
#BollingerStart(aapl_historical,20,1,1)